In [92]:
import csv
import pandas as pd
from detoxify import Detoxify
import numpy as np
import pickle
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn import svm
from sklearn import linear_model
from sklearn.linear_model import RidgeClassifier
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier

In [2]:
df_train = pd.read_csv("train_2024.csv", quoting=csv.QUOTE_NONE)
df_test = pd.read_csv("test_2024.csv", quoting=csv.QUOTE_NONE)
df_dev = pd.read_csv("dev_2024.csv", quoting=csv.QUOTE_NONE)

In [3]:
with open("train", "rb") as fp:
    train1 = pickle.load(fp)
    
with open("train_son54bin", "rb") as fp:  
    train2 = pickle.load(fp)
    
with open("dev", "rb") as fp:   
    dev = pickle.load(fp)
    
with open("test", "rb") as fp:   
    test = pickle.load(fp)

In [4]:
train = train1 + train2

In [5]:
len(train)

99000

In [6]:
N = len(train)
train_features = np.empty((N, 6))

for i, dictionary in enumerate(train):
    train_features[i] = [dictionary['toxicity'], dictionary['severe_toxicity'], dictionary['obscene'], dictionary['threat'], dictionary['insult'], dictionary['identity_attack']]

In [7]:
N = len(dev)
dev_features = np.empty((N, 6))

for i, dictionary in enumerate(dev):
    dev_features[i] = [dictionary['toxicity'], dictionary['severe_toxicity'], dictionary['obscene'], dictionary['threat'], dictionary['insult'], dictionary['identity_attack']]

In [8]:
N = len(test)
test_features = np.empty((N, 6))

for i, dictionary in enumerate(test):
    test_features[i] = [dictionary['toxicity'], dictionary['severe_toxicity'], dictionary['obscene'], dictionary['threat'], dictionary['insult'], dictionary['identity_attack']]

In [ ]:
train_values = []
for t in train:
    train_values.append(t['toxicity'])

In [ ]:
test_values = []
for t in test:
    test_values.append(t['toxicity'])

In [ ]:
dev_values = []
for t in dev:
    dev_values.append(t['toxicity'])

In [ ]:
train_features = np.array(train_values).reshape(-1,1)
test_features = np.array(test_values).reshape(-1,1)
dev_features = np.array(dev_values).reshape(-1,1)

In [9]:
dev_features.shape

(11000, 6)

In [10]:
train_labels = np.array(df_train['label'])
test_labels = np.array(df_test['label']) 
dev_labels = np.array(df_dev['label']) 

In [64]:
pca = PCA(n_components=2)
pca.fit(train_features)

PCA(n_components=2)

In [65]:
train_trans = pca.transform(train_features)
dev_trans = pca.transform(dev_features)

In [69]:
clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(n_estimators=150, random_state=1)
clf3 = AdaBoostClassifier(n_estimators=100, algorithm="SAMME",)
eclf1 = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')
eclf1.fit(train_features, train_labels)
dev_predictions = eclf1.predict(dev_features)
f1 = f1_score(dev_labels, dev_predictions)
print(f1)

0.8846625766871166
